In [10]:
# import
import numpy as np
import pandas as pd
from pathlib import Path
import chardet


In [18]:
# Create path to CSV
gaming = Path("../Scrub gaming data.csv")

In [14]:
# read in binary mode and detect encoding
# with open("../Scrub gaming data.csv", 'rb') as f:
#     result = chardet.detect(f.read())
    
# encoding = result['encoding']



In [19]:
# Read data file with Pandas
gaming_df = pd.read_csv(gaming, encoding = "ISO-8859-1")
gaming_df["whyplay"].value_counts()

whyplay
having fun                                                                          5289
improving                                                                           4927
winning                                                                             2091
relaxing                                                                             654
All of the above                                                                      26
                                                                                    ... 
Making my brain go numb                                                                1
Combination: Winning / Having fun                                                      1
Improving + Fun                                                                        1
to be occupied                                                                         1
Mental priority is Fun>Improve>Win  actual priority once in game Win>Improve>Fun       1
Name: count, 

In [20]:
gaming_df.columns

Index(['S. No.', 'Timestamp', 'GAD1', 'GAD2', 'GAD3', 'GAD4', 'GAD5', 'GAD6',
       'GAD7', 'GADE', 'SWL1', 'SWL2', 'SWL3', 'SWL4', 'SWL5', 'Game',
       'Platform', 'Hours', 'earnings', 'whyplay', 'League', 'highestleague',
       'streams', 'SPIN1', 'SPIN2', 'SPIN3', 'SPIN4', 'SPIN5', 'SPIN6',
       'SPIN7', 'SPIN8', 'SPIN9', 'SPIN10', 'SPIN11', 'SPIN12', 'SPIN13',
       'SPIN14', 'SPIN15', 'SPIN16', 'SPIN17', 'Narcissism', 'Gender', 'Age',
       'Work', 'Degree', 'Birthplace', 'Residence', 'Reference', 'Playstyle',
       'accept', 'GAD_T', 'SWL_T', 'SPIN_T', 'Residence_ISO3',
       'Birthplace_ISO3'],
      dtype='object')

In [22]:
clean_game = gaming_df[["S. No.","Game","Platform","Hours","earnings","whyplay","League","Gender","Age","Work","Degree","Residence","Playstyle","GAD_T","SWL_T","SPIN_T"]]
clean_game["Degree"].value_counts()

Degree
High school diploma (or equivalent)    8560
Bachelorÿ(or equivalent)               2687
Masterÿ(or equivalent)                  546
Ph.D.  Psy. D.  MD (or equivalent)       94
Name: count, dtype: int64